In [3]:
import networkx as nx
from pathlib import Path
import pandas as pd


def read_graph(file_path: str):
    """Read graph from file."""
    data_path = Path(file_path)
    final_nodes = pd.read_parquet(data_path / "create_final_nodes.parquet")
    final_text_units = pd.read_parquet(data_path / "create_final_text_units.parquet")
    final_relationships = pd.read_parquet(
        data_path / "create_final_relationships.parquet"
    )


    final_entities = pd.read_parquet(data_path / "create_final_entities.parquet")

    return final_nodes, final_text_units, final_relationships, final_entities
    
    

base_path = "/home/wangshu/rag/hier_graph_rag/graphrag/ragtest/output/20240813-220313/artifacts"

final_nodes, final_text_units, final_relationships, final_entities = read_graph(base_path)
final_entities.head()

,id,name,type,description,human_readable_id,graph_embedding,text_unit_ids,description_embedding
0,b45241d70f0e43fca764df95b2b81f77,GREEN TEA PRESS,ORGANIZATION,Publisher that allows copying and modification...,0,None,[ffb16092b35827ba4cb423c458ffda24],"[-0.036085054, 0.064364105, -0.18474787, -0.00..."
1,4119fd06010c494caa07f439b333f4c5,ALLEN B. DOWNEY,PERSON,"Author of the book ""Think OS""",1,None,[ffb16092b35827ba4cb423c458ffda24],"[0.026852118, 0.06175917, -0.15233214, -0.0402..."
2,d3835bf3dda84ead99deadbeac5d0d7d,"NEEDHAM, MASSACHUSETTS",GEO,Location where Green Tea Press is based,2,None,[ffb16092b35827ba4cb423c458ffda24],"[-0.02971825, 0.04920626, -0.19538683, 0.02687..."
3,077d2820ae1845bcbb1803379a3d1eae,CREATIVE COMMONS,ORGANIZATION,Organization that provides the license for sha...,3,None,[ffb16092b35827ba4cb423c458ffda24],"[0.013522442, 0.027320294, -0.15304469, -0.031..."
4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,OS,CONCEPT,"Concept related to the book ""Think OS""",4,None,[ffb16092b35827ba4cb423c458ffda24],"[-0.03602808, 0.08802482, -0.15354575, -0.0399..."


In [13]:
def read_graph_nx(file_path: str):
    """Read graph from file."""
    data_path = Path(file_path)
    final_nodes = pd.read_parquet(data_path / "create_final_nodes.parquet")
    final_text_units = pd.read_parquet(data_path / "create_final_text_units.parquet")
    final_relationships = pd.read_parquet(
        data_path / "create_final_relationships.parquet"
    )

    final_entities = pd.read_parquet(data_path / "create_final_entities.parquet")

    graph = nx.Graph()
    for _, row in final_entities.iterrows():
        graph.add_node(row["name"], **row.to_dict())

    for _, row in final_relationships.iterrows():
        graph.add_edge(row["source"], row["target"], weight=row["weight"])

    
    # 将embedding添加到图的节点属性中
    for _, row in final_entities.iterrows():
        graph.nodes[row["name"]]["embedding"] = row["description_embedding"]

    print(f"Number of nodes: {graph.number_of_nodes()}")
    print(f"Number of edges: {graph.number_of_edges()}")
    ent_embedding_sample = final_entities.iloc[0, -1]
    print(f"embedding sample:{type(ent_embedding_sample)}")
    print(f"embedding sample:{ent_embedding_sample}")
    
    return graph, final_entities, final_relationships


graph, final_entities, final_relationships = read_graph_nx(base_path)

Number of nodes: 756
Number of edges: 678
embedding sample:<class 'numpy.ndarray'>
embedding sample:[-3.60850540e-02  6.43641050e-02 -1.84747870e-01 -7.71399630e-03
  3.43688650e-02  3.36533100e-02 -3.79324100e-02  3.26789770e-02
 -8.82819900e-03 -1.08378630e-03  5.66350740e-02  9.63843800e-04
  5.48275500e-02  2.05628900e-02  2.90947480e-02  2.75712400e-02
  2.65752670e-02 -3.41265570e-02 -3.10259520e-02  2.89500450e-02
 -1.68756280e-02 -5.57917170e-02 -7.08923300e-02 -1.54252470e-02
  8.06212600e-02  1.68881430e-02 -2.85567640e-02  1.29715410e-03
 -9.61375900e-03 -8.00911400e-03  1.35355040e-03 -2.93052830e-02
  3.50370840e-03  1.77578940e-02 -1.92775100e-02  3.73806000e-04
  7.62121100e-02 -1.00401820e-02 -3.31464860e-02  5.27524130e-02
  1.82119890e-02  3.77168400e-02 -2.67769120e-02 -1.31183350e-02
  4.41824760e-02 -3.63243600e-04 -1.34502400e-02 -2.86095530e-02
  3.49678140e-02 -2.81228440e-02  3.19091940e-02 -5.39668130e-02
 -1.01337150e-02 -1.74941550e-02  4.12017070e-02  7.392